# 라이브러리

In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from math import sqrt
from tqdm import tqdm
from datetime import timedelta
from konlpy.tag import *


from sklearn.metrics import accuracy_score, f1_score, mean_squared_error, r2_score, confusion_matrix, classification_report, log_loss
from sklearn.pipeline import Pipeline 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.feature_extraction.text import TfidfVectorizer 

In [2]:
# 경고 무시
import warnings
warnings.filterwarnings('ignore')

# 종목명 선택 및 뉴스,토론방, 유튜브 데이터 통합

In [3]:
# LG화학, 삼성SDI, SK이노베이션, 고려아연, 포스코케미칼
stock_name = '삼성SDI'

In [4]:
naver_news = pd.read_csv('./data/refined_naver_news.csv', index_col = 0)
daum_news = pd.read_csv('./data/refined_daum_news.csv', index_col = 0)
naver_talks = pd.read_csv(f'./data/refined_naver_talks_{stock_name}.csv', index_col = 0)
daum_talks = pd.read_csv(f'./data/refined_daum_talks_{stock_name}.csv', index_col = 0)
youtube = pd.read_csv(f'./data/refined_youtube_{stock_name}.csv', index_col = 0)

In [5]:
# 데이터 통합
news_df = pd.concat([naver_news, daum_news, naver_talks, daum_talks ,youtube])

# 'Date' 타입이 int 이므로 datetime으로 변환
news_df['Date'] = pd.to_datetime(news_df['Date'].astype(str))

# 합쳐진 데이터들의 인덱스 재설정
news_df.sort_values('Date', ignore_index=True, inplace=True)

In [6]:
# 2021년 주식 데이터가 1월 4일부터 있어서 슬라이싱
news_df = news_df[news_df[news_df['Date']== '2021-01-04'].index[0] : ]
news_df.head(2)

,Date,Title
639,2021-01-04,비올 일본 최대 병원체인과 실펌엑스 총판계약 체결
640,2021-01-04,환율 하락 전환 1086 2 감소0 1원


# 주가 데이터

In [7]:
stock_df = pd.read_csv(f'./data/{stock_name}_주가_데이터.csv', usecols = ['일자','등락률'])
stock_df['일자'] = pd.to_datetime(stock_df['일자'])
stock_df.head(2)

,일자,등락률
0,2021-01-04,6.85
1,2021-01-05,2.24


In [8]:
start = str(stock_df.iloc[0, 0])
end = str(stock_df.iloc[-1, 0])
print(start)
print(end)

2021-01-04 00:00:00
2022-06-30 00:00:00


# 데이터 프레임 합치기

In [9]:
## 뉴스일자 조정(예측대상(주가)의 일자와 맞추기 위해)
news_df['일자'] = news_df['Date'] + timedelta(days=1)

In [10]:
df = news_df.merge(stock_df)
df.columns = [df.columns[0], df.columns[1], '주가의 날짜', '등락률' ]
df.drop_duplicates('Title', inplace = True, ignore_index = True)  # 기사제목 중복 제거
df['Title'] = df['Title'].astype(str)
print(len(df))
df.head()

429241


,Date,Title,주가의 날짜,등락률
0,2021-01-04,비올 일본 최대 병원체인과 실펌엑스 총판계약 체결,2021-01-05,2.24
1,2021-01-04,환율 하락 전환 1086 2 감소0 1원,2021-01-05,2.24
2,2021-01-04,코스피 1 03p 0 04 오른 2874 50 출발 원 달러 환율 1 2원 ...,2021-01-05,2.24
3,2021-01-04,韓증시 사상최고치 시작 개장식선 안정적 시장운영에 방점,2021-01-05,2.24
4,2021-01-04,SK바이오팜 아벨 지분 매각으로 5천500만 달러 자본이득,2021-01-05,2.24


# 감성사전 load

In [11]:
sentiment_csv = pd.read_csv('./sentiment dictionary.csv', index_col = 0)
sentiment_csv.head(3)

,pos,mid,neg
0,방긋,아직,회의적
1,상회,보통,바닥
2,신선,vs,이탈


In [12]:
pos_li = sentiment_csv['pos'].dropna().values
mid_li = sentiment_csv['mid'].dropna().values
neg_li = sentiment_csv['neg'].dropna().values

# 감성 지수 계산하는 multi_sentimental_score() 

In [13]:
def multi_sentimental_score(df):
    # 입력받은 데이터프레임 복사 및 컬럼 추가
    df_result = df.copy()
    
    df_result['Pos'] = 0
    df_result['Neg'] = 0
    df_result['Mid'] = 0
    
    # 감성 지수는 긍정 : 1, 부정 : -1, 중립 : 0, 해당 데이터 제외 : 999
    df_result['감성지수'] = 999 
    
    # 등락률에 의해 결정되는 updown    
    df_result['updown'] = 0
    df_result.loc[df_result.query('등락률 >= 1').index, 'updown'] = 1
    df_result.loc[df_result.query('등락률 <= -1').index, 'updown'] = -1
    
################################################################################################
    # 0  : 없음, 1: 있음
    # 감성 사전에 따른 텍스트 검출
    print('긍정 단어 검색중')
    for pos in tqdm(pos_li) :
        str_expr = f"Title.str.contains('{pos}')"
        df_result.loc[df_result.query(str_expr).index, 'Pos'] = 1
    
    print('부정 단어 검색중')
    for neg in tqdm(neg_li) :
        str_expr = f"Title.str.contains('{neg}')"
        df_result.loc[df_result.query(str_expr).index, 'Neg'] = 1
        
    print('중립 단어 검색중')
    for mid in tqdm(mid_li) :
        str_expr = f"Title.str.contains('{mid}')"
        df_result.loc[df_result.query(str_expr).index, 'Mid'] = 1
    
################################################################################################
    
    # 모든 종류의 단어가 검출 되면 제외
    df_result.loc[df_result.query('Pos == 1 and Neg == 1 and Mid == 1').index, '감성지수'] = 999
    
    # 중립 단어가 검출되면 중립
    df_result.loc[df_result.query('Mid == 1').index, '감성지수'] = 0
    
    # 긍정 단어만이 검출되면 긍정
    df_result.loc[df_result.query('Pos == 1 and Neg == 0 and Mid == 0').index, '감성지수'] = 1
    
    # 부정 단어만이 검출되면 부정
    df_result.loc[df_result.query('Pos == 0 and Neg == 1 and Mid == 0').index, '감성지수'] = -1
    
    # 긍정, 부정 단어가 둘 다 있으면 전 날 또는 당일 주가의 등락률을 보고 결정
    print('긍정 부정 둘 다 있는 경우 처리중')
    for i in tqdm(df_result.loc[df_result.query('Pos == 1 and Neg == 1 and Mid == 0').index].index) : 
        # 등락률에 따라 감성지수를  정하는 check
        check = 999
        
        # 해당 Title의 어제 주가가 있으면 선택
        if sum(df_result.loc[i,'Date'] - timedelta(days = 1) == stock_df['일자']) == 1 :  
            check = stock_df[stock_df['일자'] == df_result.loc[i,'Date'] - timedelta(days = 1)]['등락률'].values[0]
        
        # 어제 주가는 없지만 당일이 있으면 당일을 선택
        elif sum(df_result.loc[i,'Date'] == stock_df['일자']) == 1 :  
            check = stock_df[stock_df['일자'] == df_result.loc[i,'Date']]['등락률'].values[0]

        # 어제와 오늘의 주가도 없다면 이전의 주가를 찾아 탐색
        else :
            j = 2 
            while True :
                if sum(df_result.loc[i,'Date'] - timedelta(days = j) == stock_df['일자']) == 1 :
                    check = stock_df[stock_df['일자'] == df_result.loc[i,'Date'] - timedelta(days = j)]['등락률'].values[0]
                    break
                j += 1
        
        if check >= 1 :
            df_result.loc[i,'감성지수'] = 1
        elif check <= -1 :
            df_result.loc[i,'감성지수'] = -1
        else :
            df_result.loc[i,'감성지수'] = 0
    
################################################################################################    
    # 감성지수가 긍정 부정 둘 다 아니면 삭제
    df_result = pd.concat([df_result.loc[df_result.query('감성지수 != 999').index]], ignore_index = True)
    
    return df_result

In [14]:
updown_abs_1 = multi_sentimental_score(df)

긍정 단어 검색중


100%|████████████████████████████████████████████████████████████████████████████████| 648/648 [01:45<00:00,  6.16it/s]


부정 단어 검색중


100%|██████████████████████████████████████████████████████████████████████████████| 1001/1001 [02:58<00:00,  5.61it/s]


중립 단어 검색중


100%|██████████████████████████████████████████████████████████████████████████████████| 53/53 [00:09<00:00,  5.67it/s]


긍정 부정 둘 다 있는 경우 처리중


100%|██████████████████████████████████████████████████████████████████████████| 93455/93455 [00:57<00:00, 1638.94it/s]


In [17]:
#updown_abs_1.to_csv('./model5_updown_abs_1.csv', encoding = 'utf-8-sig')

In [15]:
#updown_abs_1 = pd.read_csv('./model5_updown_abs_1.csv', encoding='utf-8-sig', index_col=0)
updown_abs_1

,Date,Title,주가의 날짜,등락률,Pos,Neg,Mid,감성지수,updown
0,2021-01-04,비올 일본 최대 병원체인과 실펌엑스 총판계약 체결,2021-01-05,2.24,1,0,0,1,1
1,2021-01-04,환율 하락 전환 1086 2 감소0 1원,2021-01-05,2.24,0,1,0,-1,1
2,2021-01-04,코스피 1 03p 0 04 오른 2874 50 출발 원 달러 환율 1 2원 ...,2021-01-05,2.24,1,0,0,1,1
3,2021-01-04,韓증시 사상최고치 시작 개장식선 안정적 시장운영에 방점,2021-01-05,2.24,1,1,0,1,1
4,2021-01-04,SK바이오팜 아벨 지분 매각으로 5천500만 달러 자본이득,2021-01-05,2.24,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...
376992,2022-06-29,새빗켐 증권신고서 제출 코스닥 상장 본격화,2022-06-30,-6.67,1,0,0,1,-1
376993,2022-06-29,SK바사 토종 백신 1호 식약처 승인 소식에 3 대 강세,2022-06-30,-6.67,1,0,0,1,-1
376994,2022-06-29,장중시황 코스피 1 4 내린 2389선 경기 침체 우려에 약세,2022-06-30,-6.67,0,1,0,-1,-1
376995,2022-06-29,코스피 외인 기관 매도세 확대 코스닥 낙폭 축소,2022-06-30,-6.67,1,1,0,1,-1


In [16]:
updown_abs_1['updown'].value_counts()

 0    147934
 1    124255
-1    104808
Name: updown, dtype: int64

In [17]:
updown_abs_1['감성지수'].value_counts()

 1    213462
-1     92936
 0     70599
Name: 감성지수, dtype: int64

In [18]:
print(accuracy_score(updown_abs_1['updown'], updown_abs_1[ '감성지수']))

#print(accuracy_score(updown_abs_1.loc[updown_abs_1.query('감성지수 == 1').index, 'updown'],
#               updown_abs_1.loc[updown_abs_1.query('감성지수 == 1').index, '감성지수']))

#print(accuracy_score(updown_abs_1.loc[updown_abs_1.query('감성지수 == -1').index, 'updown'],
#               updown_abs_1.loc[updown_abs_1.query('감성지수 == -1').index, '감성지수']))

0.331121998318289


In [19]:
df_result = updown_abs_1.copy()
#df_result = updown_abs_0[:5000]

In [20]:
df_result

,Date,Title,주가의 날짜,등락률,Pos,Neg,Mid,감성지수,updown
0,2021-01-04,비올 일본 최대 병원체인과 실펌엑스 총판계약 체결,2021-01-05,2.24,1,0,0,1,1
1,2021-01-04,환율 하락 전환 1086 2 감소0 1원,2021-01-05,2.24,0,1,0,-1,1
2,2021-01-04,코스피 1 03p 0 04 오른 2874 50 출발 원 달러 환율 1 2원 ...,2021-01-05,2.24,1,0,0,1,1
3,2021-01-04,韓증시 사상최고치 시작 개장식선 안정적 시장운영에 방점,2021-01-05,2.24,1,1,0,1,1
4,2021-01-04,SK바이오팜 아벨 지분 매각으로 5천500만 달러 자본이득,2021-01-05,2.24,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...
376992,2022-06-29,새빗켐 증권신고서 제출 코스닥 상장 본격화,2022-06-30,-6.67,1,0,0,1,-1
376993,2022-06-29,SK바사 토종 백신 1호 식약처 승인 소식에 3 대 강세,2022-06-30,-6.67,1,0,0,1,-1
376994,2022-06-29,장중시황 코스피 1 4 내린 2389선 경기 침체 우려에 약세,2022-06-30,-6.67,0,1,0,-1,-1
376995,2022-06-29,코스피 외인 기관 매도세 확대 코스닥 낙폭 축소,2022-06-30,-6.67,1,1,0,1,-1


# 트레인 테스트 분리

In [21]:
# X : title, y : price
def data_split(X, y) :
    
    # 학습셋, 테스트셋 분리
    X_list = X.tolist()
    y_list = y.tolist()
    
    X_train, X_test, y_train, y_test = train_test_split(X_list, y_list, shuffle = True, test_size = 0.2)
    
    return X_train, X_test, y_train, y_test

In [22]:
X_train, X_test, y_train, y_test = data_split(df_result['Title'], df_result['감성지수'])

In [23]:
print(len(X_train))
print(len(X_test))

301597
75400


# 모델

### tokenizer
- TF-IDF는 단어의 빈도와 역 문서 빈도를 사용하여 DTM 내의 각 단어들마다 중요한 정도를 가중치로 주는 기법이다. 
- TF-IDF : CountVectorizer 와 TfidfTransformer를 합쳐 놓은 TfidfVectorizer를 사용

### LogisticRegression

> parameters 구성

1. penalty
    - `['l1','l2','elasticnet','none']`
    - 기본값 : l2
    - L2: Ridge, 일반적으로 사용 (default)
    - L1: LASSO, 변수가 많아서 줄여야할 때 사용, 모델의 단순화 및 해석에 용이
    - L2는 L1 보다 가중치 규제에 좀 더 효과적

2. C 
    - 규제의 강도를 조절하는 파라미터
    - 값이 클수록 규제가 약해지고, 값이 작을수록 규제가 강해진다.
    - 로그스케일로 적절히 조절하여 최상의 성능을 내는 값을 찾도록한다.
    

   
3. solver
    - 최적화 문제를 풀기 위한 알고리즘을 선택하는 파라미터
    - `['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']`
    - 기본값 : lbfgs
    - 작은 데이터셋 -> liblinear
    - 큰 데이터셋 -> sag 또는 saga
    - 다중 클래스 문제의 경우 'newton-cg', 'sag', 'saga' 및 'lbfgs'만 다항 손실을 처리
    - liblinear는 일대일 방식
    - 최적화문제 -> 비용 또는 손실 함수를 최소화하는 최상의 가중치/모형함수를 찾는 것
    - solver마다 지원하는 패널티가 달라 조정해야함.
        
    - newton-cg : [‘l2’, ‘none’]
    - lbfgs : [‘l2’, ‘none’]
    - liblinear : [‘l1’, ‘l2’]
    - sag : [‘l2’, ‘none’]
    - saga : [‘elasticnet’, ‘l1’, ‘l2’, ‘none’]
    
    
__기본 Logistic Regression__
    
    solver : 최적화 문제에 사용할 알고리즘
    
    iblinear : L1, L2 모두 지원, 작은 데이터에 적합한 알고리즘
    
    sag : L2만 지원, 확률적경사하강법을 기반으로 대용량 데이터에 적합한 알고리즘
    saga : L1, L2 모두 지원, 확률적경사하강법을 기반으로 대용량 데이터에 적합한 알고리즘
    
    
    newton-cg : L2만 지원, 멀티클래스의 분류 모델에 쓰이는 알고리즘
    lbfgs : L2만 지원, 멀티클래스의 분류 모델에 쓰이는 알고리즘
    L1, L2에 대한 설명 : L1, L2

4. max_iter
    - solver에 의해 진행되는 수렴을 위한 반복의 최대 횟수를 지정한다.
    - 데이터에 따라, solver에 따라 수렴이 오래걸리는 경우가 있어 조절 필요
    - 기본값 : 100
   
5. multi_class
    - 다중클래스분류 문제의 상황에서 어떤 접근방식을 취할지 결정한다.
    - ovr : one vs rest 방식으로 각 클래스 레이블에 대해 이진 분류 문제를 푼다.
    - multinomial : 각 클래스에 대한 softmax 확률값, 크로스엔트로피 계산을 통해 one hot 타겟 벡터를 생성
    - 기본값 : auto

### GridSearchCV

fit(X, y): 학습

- predict(X): 분류-추론한 class, 회귀-추론한 값

- predict_proba(X): 분류문제에서 class별 확률을 반환 

- best_params_: 가장 좋은 성능을 낸 parameter 조합을 반환

- best_estimator_: 가장 좋은 성능을 낸 모델 반환 

- best_score_: 가장 좋은 점수 반환 


# 토큰화 및 로지스틱 회귀 적용

- 먼저 로지스틱 회귀의 그리드 서치를 위해 데이터에 토큰화 적용

In [24]:
# 형태소 분석을 위한 함수
def tokenizer(text):
    okt = Okt()
    return okt.morphs(text)

In [25]:
tokenize_X_train = TfidfVectorizer(tokenizer = tokenizer).fit_transform(X_train)
tokenize_X_test = TfidfVectorizer(tokenizer = tokenizer).fit_transform(X_test)

# 로지스틱 회귀 그리드 서치

In [26]:
def parameter_serach(penalty, solvers) :
    best_score = 0
    Cs = [0.001, 0.01, 0.1, 1, 10, 100]
        
    kfold = KFold(3, shuffle = True)

    for s in solvers :
        scores_mean = []
        for C in tqdm(Cs) :
            lm = LogisticRegression(penalty = penalty, C = C, solver = s, multi_class = 'multinomial')
            scores = cross_val_score(lm, tokenize_X_train, y_train, cv = kfold)

            score = np.mean(scores)
            scores_mean.append(score)

            if score > best_score:
                best_score = score
                best_parameters = {'penalty' : penalty, 'C': C, 'solver': s, 'best_score' : best_score}

    return best_parameters 

In [27]:
# saga : 쓸 수 있지만 C값이 1 이상부터 시간이 너무 오래 걸림(몇분 단위)
#l1_best_parameters = parameter_serach('l1', ['saga'])

In [28]:
l2_best_parameters = parameter_serach('l2', ['lbfgs', 'newton-cg',  'sag', 'saga'])

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [02:19<00:00, 23.21s/it]


#### 로지스틱 회귀의 최적 파라미터 확인

In [29]:
# print(f"penalty\t\t:\t{l1_best_parameters['penalty']}")
# print(f"C\t\t:\t{l1_best_parameters['C']}")
# print(f"solver\t\t:\t{l1_best_parameters['solver']}")
# print(f"best_score\t:\t{l1_best_parameters['best_score']}")

In [30]:
print(f"penalty\t\t:\t{l2_best_parameters['penalty']}")
print(f"C\t\t:\t{l2_best_parameters['C']}")
print(f"solver\t\t:\t{l2_best_parameters['solver']}")
print(f"best_score\t:\t{l2_best_parameters['best_score']}")

penalty		:	l2
C		:	10
solver		:	newton-cg
best_score	:	0.8456450165683815


In [31]:
# if l1_best_parameters['best_score'] > l2_best_parameters['best_score'] :
#     best_param = l1_best_parameters
# else :
#     best_param = l2_best_parameters

In [32]:
best_param = l2_best_parameters
best_param 

{'penalty': 'l2',
 'C': 10,
 'solver': 'newton-cg',
 'best_score': 0.8456450165683815}

# 파이프 라인 

- 위쪽부터 진행한 토큰화, 벡터화, 로지스틱 회귀를 파이프라인으로 구성
- 로지스틱 회귀는 그리드 서치를 통한 최적의 파라미터를 갖고 진행

In [33]:
# lowercase : 소문자로 반환 여부 lowercase = False, 
# tokenizer : 토크나이저 지정
tfidf = TfidfVectorizer(tokenizer = tokenizer)

In [34]:
logistic = LogisticRegression(penalty = best_param['penalty'],
                              C = best_param['C'],
                              solver = best_param['solver'],
                              multi_class = 'multinomial')

In [35]:
pipeline = Pipeline([('tfidf',tfidf), ('classfier', logistic)], verbose = True) 

In [36]:
pipeline.fit(X_train, y_train)

[Pipeline] ............. (step 1 of 2) Processing tfidf, total= 5.8min
[Pipeline] ......... (step 2 of 2) Processing classfier, total= 1.2min


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function tokenizer at 0x000002425AF93820>)),
                ('classfier',
                 LogisticRegression(C=10, multi_class='multinomial',
                                    solver='newton-cg'))],
         verbose=True)

#### 교차검증

In [37]:
score = cross_val_score(pipeline, X_train, y_train, cv = KFold(3, shuffle = True), scoring='f1_micro')

[Pipeline] ............. (step 1 of 2) Processing tfidf, total= 7.2min
[Pipeline] ......... (step 2 of 2) Processing classfier, total=  37.3s
[Pipeline] ............. (step 1 of 2) Processing tfidf, total= 4.9min
[Pipeline] ......... (step 2 of 2) Processing classfier, total=  30.8s
[Pipeline] ............. (step 1 of 2) Processing tfidf, total= 4.3min
[Pipeline] ......... (step 2 of 2) Processing classfier, total=  29.1s


In [38]:
print(score)
print(f"모델의 정확도\t:\t{score.mean()}")

[0.84636885 0.84548204 0.84530299]
모델의 정확도	:	0.845717959227397


In [39]:
y_df = df_result.loc[df_result.query(f'Title == {X_test}').index, 'updown'].values

In [42]:
y_pred = pipeline.predict(X_test)

In [41]:
y_proba = pipeline.predict_proba(X_test)

#  평가

In [46]:
print(f'accuracy \t:\t {accuracy_score(y_df, y_pred)}')

#'micro', 'macro', 'weighted'
print(f'f1_score\t:\t {f1_score(y_df, y_pred, average = "micro")}')
print(f'f1_score\t:\t {f1_score(y_df, y_pred, average = "macro")}')
print(f'f1_score\t:\t {f1_score(y_df, y_pred, average = "weighted")}')

print(f'R2 \t\t:\t {r2_score(y_df, y_pred)}')

print(f'mse \t\t:\t {mean_squared_error(y_df, y_pred)}')

print(f'rmse \t\t:\t {mean_squared_error(y_df, y_pred, squared=False)}')

print(f'log_loss\t:\t {log_loss(y_df, y_proba)}')

accuracy 	:	 0.3270954907161804
f1_score	:	 0.3270954907161804
f1_score	:	 0.3053784314120483
f1_score	:	 0.30206882765602544
R2 		:	 -1.3339960883826585
mse 		:	 1.4090981432360743
rmse 		:	 1.1870543977577752
log_loss	:	 2.9290084342937814


In [47]:
print(confusion_matrix(y_pred, y_df))
print(classification_report(y_pred, y_df))

[[ 5407  7553  6295]
 [ 3274  4661  3949]
 [12208 17458 14595]]
              precision    recall  f1-score   support

          -1       0.26      0.28      0.27     19255
           0       0.16      0.39      0.22     11884
           1       0.59      0.33      0.42     44261

    accuracy                           0.33     75400
   macro avg       0.33      0.33      0.31     75400
weighted avg       0.44      0.33      0.35     75400



# 모델 저장 및 사용

In [48]:
def save_model(model):
    with open('./[Model5]pipe_multi.dat', 'wb') as fp:     # 쓰기
        pickle.dump(model, fp)
    print('저장완료')     # 학습된 모델 저장 완료

In [49]:
save_model(pipeline)

저장완료


In [50]:
with open('./[Model5]pipe_multi.dat','rb') as fp:     # 읽기
    pipeline = pickle.load(fp)

# Inference

In [51]:
# 모델 사용 함수

def model_prediction():  
    # 객체를 복원, 저장된 모델 불러오기
    with open('./[Model5]pipe_multi.dat','rb') as fp:     # 읽기
        pipe = pickle.load(fp)
    while True :
        text = input('뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : \n')
        example = [text]
        # 예측 정확도
        r1 = np.max(pipe.predict_proba(example) * 100)     # 확률값을 구해서 *100
        # 예측 결과
        r2 = pipe.predict(example)[0]     # 긍정(1), 중립(-1), 중립(0)

        if text == 'q':
            print("예측을 종료합니다.")
            break

        if r2 == 1 :
            print(f'{stock_name} 주가가 상승할 것으로 예상됩니다.')
        elif r2 == -1 :
            print(f'{stock_name} 주가가 하락할 것으로 예상됩니다.')
        else : 
            print(f'\n모르겠어요.....')
        print('확률 : %.3f' % r1)
        print('------------------------------------------------\n')

In [52]:
model_prediction()

뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : 
[줌인 이종목] 현대중공업, 실적 증가 기대에 껑충
삼성SDI 주가가 상승할 것으로 예상됩니다.
확률 : 97.992
------------------------------------------------

뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : 
닷새만에 반등했지만…불안한 개미들, 7100억 차익실현
삼성SDI 주가가 하락할 것으로 예상됩니다.
확률 : 68.396
------------------------------------------------

뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : 
2차전지株, 실적 부진해도 상승하는 까닭
삼성SDI 주가가 하락할 것으로 예상됩니다.
확률 : 72.143
------------------------------------------------

뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : 
포토] 환율 13년여 만에 최고치, 오늘의 증시는?
삼성SDI 주가가 상승할 것으로 예상됩니다.
확률 : 91.467
------------------------------------------------

뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : 
증시 다시 폭풍속으로..연고점 경신한 환율
삼성SDI 주가가 하락할 것으로 예상됩니다.
확률 : 48.787
------------------------------------------------

뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : 
q
예측을 종료합니다.



```
[줌인 이종목] 현대중공업, 실적 증가 기대에 껑충
닷새만에 반등했지만…불안한 개미들, 7100억 차익실현
주식거래 뚝…"증권사, 2분기 실적 쇼크"
'현대차 효과' 부품株에도 볕드나.. 미래형 자동차 양산 기대감 호재 [현대차, 29년만에 국내공장 신설]
2차전지株, 실적 부진해도 상승하는 까닭
블랙록 '극심한 변동성' 경고.."지금은 저가매수도 하지 말라"
[유럽개장] 일제히 하락세.. DAX 1.07%↓ 

[포토] 환율 13년여 만에 최고치, 오늘의 증시는?

증시 다시 폭풍속으로..연고점 경신한 환율



```